<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/%E5%A4%9AChatBot%E6%8A%BD%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 对特定小说抽取chatbot
Implemented by 李鲁鲁

- [x] 读取总结之后输出的jsonl
- [x] 总结所有角色出现的频率，可视化
- [x] 想个办法截取特定角色
- [x] 如果太长则切一下

TODO 米唯实：

- [x] 把role_name和几个参数的设计移动到脚本前面来
- [x] 支持target_names设置多个人物
- [x] 如果不设置，自动整理出现最多的非scene的3个人物
- [x] 把抽取后的三个文本，整理成txt和jsonl的那个notebook上传到github的文件夹
- [x] 把整理后的这个脚本 ，也上传到这个文件夹
- [ ] 在readme，详细说明小说抽取、整合、和chatbot提取的用法

https://github.com/LC1332/Chat-Haruhi-Suzumiya/tree/main/characters/novel_collecting

这里



## 载入包

In [14]:
!pip install -q tiktoken
import os
import zipfile
import tiktoken
import json

## 参数设置

In [2]:
# 参数设置

# 支持跨越多少行寻找目标角色，也即控制段内行间距不超过该值
max_find_lines = 10

max_token_num = 500

# target_role支持 空字符串(默认前三个)或者List of string 如果出错默认保存第一个
target_role = ['郭靖', "欧阳锋"]

# 输入文件路径
input_name = '/content/shediaoyingxiongzhuan.jsonl'

# 保存路径
savepath = '/content/texts'
!rm -rf /content/texts
os.makedirs(savepath, exist_ok=True)
# zip_path = '/content/linghuchong_text.zip'

## 对小说进行切分和统计

In [3]:
# input_name = '/content/reorganized_xiaoaojianghu.jsonl'
# 用utf8编码为我读取jsonl文件，并且把每一行解析成一个list of json, data
# 请用python为我实现

# # 输入文件路径
# input_name = '/content/shediaoyingxiongzhuan.jsonl'

enc = tiktoken.get_encoding("cl100k_base")

data_in_chunk = []
with open(input_name, encoding='utf8') as f:
    for line in f:
        data_in_chunk.append(json.loads(line))

data = []

for chunk in data_in_chunk:
    for d in chunk:
        data.append(d)

print(len(data))
for i,d in enumerate(data):
    if d['role'] == 'scene':
        first_scene_id = i
        break
print(first_scene_id)

14956
2


*data是一个list* of json，里面有一个字段是role

为我统计role里面各个字段出现的频率，并输出出现最多的30个角色和对应的频率

In [4]:
from collections import Counter

role_counts = Counter()
for line in data:
    role = line['role']
    role_counts[role] += 1

common_roles = role_counts.most_common(30)

status = 0

role_name = []

for role, count in common_roles:
    status = status + 1
    if role != 'scene':
        role_name.append(role)
    if status % 3 == 0:
        print(role, count)
    else:
        print(role, count, end=' ')
sorted_roles = sorted(common_roles, key=lambda x: x[1], reverse=True)
sorted_roles_clear = [role[0] for role in sorted_roles if role[0] != "scene"]

scene 5129 郭靖 2002 黄蓉 1788
洪七公 463 欧阳锋 337 黄药师 307
周伯通 284 丘处机 224 柯镇恶 180
朱聪 172 欧阳克 164 梅超风 152
穆念慈 150 杨康 150 完颜康 125
裘千仞 120 铁木真 110 杨铁心 101
包惜弱 98 陆冠英 98 成吉思汗 94
韩小莹 82 瑛姑 80 完颜洪烈 75
彭连虎 73 王处一 69 - 64
一灯大师 64 傻姑 56 程瑶迦 56


## 获取需要提取的人物列表

In [5]:
# 名为role_best的list of str，其按升序记录了所有的角色，
# 请编写一个Python程序，其判断target_role的类型，如果是空字符串或者None，就将role_best前三行提取到一个名为role_extract的list of string；
# 如果target_role为一个list of string，则逐个判断其中的每一个字符串是否在role_best中出现，如果出现，则提取到一个名为role_extract的list of string，否则打印一行错误信息，
# 在遍历完target_role后，如果role_extract不足三个，就用role_best从前往后补充至三个

def extract_roles(role_best, target_role):
    role_extract = []

    if target_role is None or target_role == ""or target_role == [""]:
        role_extract = role_best[:3]
    elif isinstance(target_role, list):
        for role in target_role:
            if role in role_best:
                role_extract.append(role)
            else:
                print(f"Error: Role '{role}' not found in role_best!")
    else:
        print("Error: Invalid target_role type!")
        return

    if len(role_extract) < 1:
        additional_roles = role_best[:1 - len(role_extract)]
        role_extract.extend(additional_roles)

    return role_extract
role_extract = extract_roles(sorted_roles_clear, target_role)

## 可视化代码，Chatbot抽取时可以不运行

In [24]:
from IPython.core.interactiveshell import default_banner
role_in_time = []

dialogue_in_time = []



for d in data:
    role = d['role']
    if role == 'scene':
        continue

    if role in role_name:

        role_in_time.append(role)
        dialogue_in_time.append(d['text'])


import plotly.express as px

# 创建映射字典,角色名字到编号
name_to_num = {name:i for i,name in enumerate(role_name)}

# 将角色名字序列转换为编号序列
role_index = [name_to_num[name] for name in role_in_time]

# 生成不同颜色的颜色序列
colors = px.colors.qualitative.Dark24
while len(colors) < len(role_name):
    colors = colors + colors


# 绘制散点图
fig = px.scatter(
    x=list(range(len(role_in_time))),
    y=role_index,
    color=[colors[i] for i in role_index],
    hover_data={'t': dialogue_in_time}
)

# 设置y轴为角色名字
fig.update_yaxes(ticktext=role_name, tickvals=list(range(len(role_name))))


# 设置坐标轴范围
fig.update_xaxes(range=[0, len(role_in_time)])
# fig.update_yaxes(range=[0, len(role_name)])

# 设置坐标轴标题
fig.update_layout(
    xaxis_title='时间',
    yaxis_title='角色'
)

fig.show()

新的逻辑非常简单，我们设置一个max_find_lines = 10

支持跨越多少行寻找目标角色。

如果找不到，就给截断了。然后对于连续的，我做一个切分。

## 联系角色和旁白

In [6]:
def output_scene_chat_id(data, target_role_single):

  chat_ids = []

  # 先寻找所有出现角色的节点
  for i,d in enumerate(data):
      if d['role'] == target_role_single:
          chat_ids.append(i)

  previous_scene_ids = []

  # 对于每一个chat_ids，向前寻找scene的节点
  for chat_id in chat_ids:
      ans = first_scene_id
      for j in range(chat_id, first_scene_id,-1):
          if data[j]['role'] == 'scene':
              ans = j
              break
      previous_scene_ids.append(ans)
  return chat_ids, previous_scene_ids


## 分块，决定texts组织内容

chat_ids是一个list of int。代表chat出现的顺序，是个严格升序的序列

我希望找到chat_ids中，所有的元素之间间隔不大于max_find_lines = 10连续子串，用list of list of int的形式，保存到chat_ids_in_chunk

例子输入
1,2,4,100,110,120,555
例子输出
[1,2,4],[100,110,120],[555]

请用python为我实现

In [7]:
def divide_chats2chunks(chat_ids, previous_scene_ids):
  chat_ids_in_chunk = []
  current_chunk = []

  for chat_id in chat_ids:
      if not current_chunk:
          current_chunk.append(chat_id)
          continue

      if chat_id - current_chunk[-1] <= max_find_lines:
          current_chunk.append(chat_id)
      else:
          chat_ids_in_chunk.append(current_chunk)
          current_chunk = [chat_id]

  if current_chunk:
      chat_ids_in_chunk.append(current_chunk)

  # print(chat_ids_in_chunk[0])

  chat_id2previous_scene_id = {}

  for previous, chat_id in zip(previous_scene_ids, chat_ids):
      chat_id2previous_scene_id[chat_id] = previous
      if previous > 0:
          if data[previous-1]['role'] != target_role:
              chat_id2previous_scene_id[chat_id] -= 1
  return chat_ids_in_chunk, chat_id2previous_scene_id

# chat_ids的分块， chat_id对应的旁白id

## 组织texts

计算一下每一句所花的token数量

In [8]:
def count_token( my_str ):
    return len(enc.encode(my_str))
def data2str( data ):
    role = data['role']
    if role in ['旁白', '', 'scene','Scene','narrator' , 'Narrator']:
        return 'scene:' + data['text']
    else:
        return role + ':「' + data['text'] + '」'


我们现在需要把这东西变成最终的texts文本

In [9]:
def id2texts(data, chat_ids_in_chunk, chat_id2previous_scene_id):
  line_token = [count_token(data2str(d)) for d in data]
  from ast import Break
  final_chunks = []

  print_count = 0


  appended_key = []

  for chunk in chat_ids_in_chunk:
      N = len(chunk)

      current_i = 0

      while current_i < N-1:



          consider_chat_id = chunk[current_i]

          previous_scene_id = chat_id2previous_scene_id[consider_chat_id]

          #保底
          withdraw_start = previous_scene_id
          withdraw_end = consider_chat_id

          current_count = sum(line_token[previous_scene_id:consider_chat_id+1])
          while current_count < max_token_num and current_i < N-1:
              consider_end = chunk[current_i + 1]
              consider_count = sum(line_token[previous_scene_id:consider_end+1])
              if consider_count < max_token_num:
                  current_count = consider_count
                  withdraw_start = previous_scene_id
                  withdraw_end = consider_end
                  current_i += 1
              else:
                  break

          # print_count += 1

          # print(withdraw_start, end = ' ')
          # if print_count % 5 == 0:
          #     print()

          if withdraw_end+1 not in appended_key:
              appended_key.append(withdraw_end+1)
              chunk_str = ''
              for i in range(withdraw_start, withdraw_end+1):
                  chunk_str += data2str(data[i]) + '\n'

              final_chunks.append(chunk_str)

          current_i += 1
  return appended_key, final_chunks




## 保存成不同的形式

In [12]:
def save_chunk2zip(savepath, save_title, final_chunks):
  os.makedirs(savepath, exist_ok=True)
  for i in range(0,len(final_chunks)):
    my_str = final_chunks[i]
    with open(savepath + f'/text_{i}.txt', 'w', encoding='utf-8') as f:
      f.write(my_str)
  zip_path = "/content/" + save_title + "_text.zip"
  with zipfile.ZipFile(zip_path, 'w') as zipf:
    for filename in os.listdir(savepath):
      zipf.write(savepath + "/" + filename)

  print('Zipped folder saved to', zip_path)


## 根据 role_extract 中的内容组织

In [13]:
for role_cur_name in role_extract :
  chat_ids, previous_scene_ids = output_scene_chat_id(data, role_cur_name)
  chat_ids_in_chunk, chat_id2previous_scene_id = divide_chats2chunks(chat_ids, previous_scene_ids)
  appended_key, final_chunks = id2texts(data, chat_ids_in_chunk, chat_id2previous_scene_id)
  save_chunk2zip(savepath+"/"+role_cur_name, role_cur_name, final_chunks) #如果你想修改保存的zip名称，请修改本函数的第二个参数save_title

Zipped folder saved to /content/郭靖_text.zip
Zipped folder saved to /content/欧阳锋_text.zip
